# resize

In [2]:
!pip install split-folders matplotlib

  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)


In [4]:
!pip install opencv-python

     |████████████████████████████████| 30.0 MB 998 kB/s eta 0:00:01


In [5]:
import splitfolders

import os

import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf

import pickle

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [16]:
dataset = {'damaged_lot': []}
folder_path = 'damaged_lot/'
dest_folder_path = 'resized_224_224'

# 이미지와 라벨 리스트에 담기
for filename in os.listdir(folder_path):
    dataset['damaged_lot'].append(folder_path + filename)

dataset

{'damaged_lot': ['damaged_lot/10447.jpg',
  'damaged_lot/06231.jpg',
  'damaged_lot/01238.jpg',
  'damaged_lot/01576.jpg',
  'damaged_lot/03163.jpg',
  'damaged_lot/02269.jpg',
  'damaged_lot/03188.jpg',
  'damaged_lot/10692.jpg',
  'damaged_lot/06147.jpg',
  'damaged_lot/01628.jpg',
  'damaged_lot/04234.jpg',
  'damaged_lot/03215.jpg',
  'damaged_lot/01402.jpg',
  'damaged_lot/05116.jpg',
  'damaged_lot/00907.jpg',
  'damaged_lot/08396.jpg',
  'damaged_lot/10283.jpg',
  'damaged_lot/01371.jpg',
  'damaged_lot/10268.jpg',
  'damaged_lot/06608.jpg',
  'damaged_lot/04989.jpg',
  'damaged_lot/04180.jpg',
  'damaged_lot/01946.jpg',
  'damaged_lot/10888.jpg',
  'damaged_lot/00457.jpg',
  'damaged_lot/06026.jpg',
  'damaged_lot/01588.jpg',
  'damaged_lot/01577.jpg',
  'damaged_lot/03360.jpg',
  'damaged_lot/10452.jpg',
  'damaged_lot/00127.jpg',
  'damaged_lot/00133.jpg',
  'damaged_lot/10320.jpg',
  'damaged_lot/10446.jpg',
  'damaged_lot/00125.jpg',
  'damaged_lot/06226.jpg',
  'damaged_lo

In [17]:
len(dataset['damaged_lot'])

715

In [23]:
for label, filenames in dataset.items():
    for filename in filenames:
        try:
            img = cv2.imread(filename)

            # 이미지의 x, y가 224이 넘을 경우 작게해주기
            percent = 1
            if(img.shape[1] > img.shape[0]) :       # 이미지의 가로가 세보다 크면 가로를 640으로 맞추고 세로를 비율에 맞춰서
                percent = 224/img.shape[1]
            else :
                percent = 224/img.shape[0]

            img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
                    # 이미지 범위 지정
            y,x,h,w = (0,0,img.shape[0], img.shape[1])

            # 그림 주변에 검은색으로 칠하기
            w_x = (224-(w-x))/2  # w_x = (224 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
            h_y = (224-(h-y))/2

            if(w_x < 0):         # 크기가 -면 0으로 지정.
                w_x = 0
            elif(h_y < 0):
                h_y = 0

            M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
            img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  
        
            # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
            dest_file_name = '{2}/{0}/{1}'.format(label, filename.split("/")[-1], dest_folder_path)
            cv2.imwrite(dest_file_name, img_re)
            print(dest_file_name, 'is Writed...')

        except Exception:
            print(filename, 'is ERROR...')

resized_224_224/damaged_lot/10447.jpg is Writed...
resized_224_224/damaged_lot/06231.jpg is Writed...
resized_224_224/damaged_lot/01238.jpg is Writed...
resized_224_224/damaged_lot/01576.jpg is Writed...
resized_224_224/damaged_lot/03163.jpg is Writed...
resized_224_224/damaged_lot/02269.jpg is Writed...
resized_224_224/damaged_lot/03188.jpg is Writed...
resized_224_224/damaged_lot/10692.jpg is Writed...
resized_224_224/damaged_lot/06147.jpg is Writed...
resized_224_224/damaged_lot/01628.jpg is Writed...
resized_224_224/damaged_lot/04234.jpg is Writed...
resized_224_224/damaged_lot/03215.jpg is Writed...
resized_224_224/damaged_lot/01402.jpg is Writed...
resized_224_224/damaged_lot/05116.jpg is Writed...
resized_224_224/damaged_lot/00907.jpg is Writed...
resized_224_224/damaged_lot/08396.jpg is Writed...
resized_224_224/damaged_lot/10283.jpg is Writed...
resized_224_224/damaged_lot/01371.jpg is Writed...
resized_224_224/damaged_lot/10268.jpg is Writed...
resized_224_224/damaged_lot/066

# augmentation

In [19]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [20]:
folder_path = 'resized_224_224'

In [24]:
# for label in  os.listdir(folder_path)[1:2]:
label = 'damaged_lot'
label_path = folder_path + '/' + label + '/'
if os.path.isdir(label_path):
    for filename in os.listdir(label_path): 
        filepath = label_path + filename

        img = load_img(filepath)
        # img 출력
        # plt.imshow(img)
        # break
        x = img_to_array(img)
        # x.shape 출력
        # print(x.shape)
        # break
        x = x.reshape((1,) + x.shape)

        i = 0
        # flow : augmentation 함수
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=label_path, save_prefix=label, save_format='jpg'):
            i += 1
            if i > 23:
                break  

# train, val, test 폴더 나눠주기

In [25]:
splitfolders.ratio('resized_224_224', output='dataset_224_224', seed=77, ratio=(0.6, 0.2, 0.2))

Copying files: 28590 files [00:14, 1946.28 files/s]


# x_train, y_train, x_val, y_val, x_test, y_test 불러오기

In [26]:
folder_path = 'dataset_224_224/train'

dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):

    sub_path = folder_path+'/'+label+'/'
    if os.path.isdir(sub_path):
        dataset[label] = []

        for filename in os.listdir(sub_path):
            dataset[label].append(sub_path+filename)

len(dataset)

3

In [29]:
label2index = {'color_refined' : 0, 'damaged_little' : 1 , 'damaged_lot' : 2}

In [30]:
x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_train.append(image)
        y_train.append(label2index[label]) # label을 index로 변경

In [35]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = x_train.astype('int8')
print(x_train.shape, y_train.shape)

(17154, 224, 224, 3) (17154,)


In [31]:
folder_path = 'dataset_224_224/val'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

len(dataset)

3

In [32]:
x_val, y_val = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_val.append(image)
        y_val.append(label2index[label]) # label을 index로 변경

In [33]:
x_val, y_val= np.array(x_val), np.array(y_val)
x_val = x_val.astype('int8') 
print(x_val.shape, y_val.shape)

(5718, 224, 224, 3) (5718,)


In [34]:
folder_path = 'dataset_224_224/test'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

len(dataset)

3

In [36]:
x_test, y_test = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_test.append(image)
        y_test.append(label2index[label]) # label을 index로 변경

In [37]:
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = x_test.astype('int8') 
print(x_test.shape, y_test.shape)

(5718, 224, 224, 3) (5718,)


# zero centering

In [41]:
def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0)

zero_mean_img = zero_mean(x_train)

print(zero_mean_img.shape)
zero_mean_img = zero_mean_img.astype('int8') 

(224, 224, 3)


In [42]:
x_train -= zero_mean_img
x_val -= zero_mean_img
x_test -= zero_mean_img
print(x_train.shape, x_val.shape, x_test.shape)

(17154, 224, 224, 3) (5718, 224, 224, 3) (5718, 224, 224, 3)


# save pickle

In [39]:
folder_path = 'dataset_224_224/'

In [40]:
with open(folder_path+'x_train.pickle', 'wb') as f:
    pickle.dump(x_train, f)

with open(folder_path+'y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)

with open(folder_path+'x_val.pickle', 'wb') as f:
    pickle.dump(x_val, f)

with open(folder_path+'y_val.pickle', 'wb') as f:
    pickle.dump(y_val, f)

with open(folder_path+'x_test.pickle', 'wb') as f:
    pickle.dump(x_test, f)

with open(folder_path+'y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [ ]:
with open(folder_path+'x_train_zero.pickle', 'wb') as f:
    pickle.dump(x_train, f)

with open(folder_path+'y_train_zero.pickle', 'wb') as f:
    pickle.dump(y_train, f)

with open(folder_path+'x_val_zero.pickle', 'wb') as f:
    pickle.dump(x_val, f)

with open(folder_path+'y_val_zero.pickle', 'wb') as f:
    pickle.dump(y_val, f)

with open(folder_path+'x_test_zero.pickle', 'wb') as f:
    pickle.dump(x_test, f)

with open(folder_path+'y_test_zero.pickle', 'wb') as f:
    pickle.dump(y_test, f)